## 過去のNotebook

[日本語][EDA] JP Tokyo Stock データ概要を確認 その1 https://www.kaggle.com/code/tatsuyafujii/eda-jp-tokyo-stock-1 <br>
[日本語][EDA] JP Tokyo Stock データ概要を確認 その2[移動平均線等] https://www.kaggle.com/code/tatsuyafujii/eda-jp-tokyo-stock-2 <br>


In [ ]:
!pip install japanize_matplotlib

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import seaborn as sns
import japanize_matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def MA(series, window):
    return series.rolling(window).mean()

def DMA(series, window=25):
    return series/MA(series, window) - 1

## メインデータ

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv", parse_dates=["Date"])
print(prices.shape)
prices.head()

## 概観

In [ ]:
## 翌日、翌々日の終値を取得（Target候補）
df = prices.copy()
df["Close_adj"] = df.groupby("SecuritiesCode").apply(lambda d:d["Close"]/d["AdjustmentFactor"].cumprod().shift().fillna(1)).reset_index("SecuritiesCode",drop=True)
df["翌日終値"] = df.groupby("SecuritiesCode")["Close_adj"].shift(-1)
df["翌々日終値"] = df.groupby("SecuritiesCode")["Close_adj"].shift(-2)
df["R"] = (df["翌々日終値"] - df["翌日終値"]) / df["翌日終値"]
(df["R"]-df["Target"]).describe()

In [ ]:
## ある一つの銘柄に注目して確認
d = df[df["SecuritiesCode"]==4923].copy()

sns.lineplot(x=d.Date, y=d.R, label="収益率")
sns.lineplot(x=d.Date, y=d.Target, label="Target")
plt.show()

## ボリンジャーバンド
直近のボラも考慮し、標準偏差で乖離率を考えるのがボリンジャーバンド。<br>
ただし、ボリンジャーバンドでは、逆張りより順張りで用いられる（通常は起こらないことが起こっているため、ボックス相場からトレンドが出来たと判断）<br>
ボリンジャーバンドが収縮し、横ばい（ボックス相場）のあとのブレークを検知するために利用。<br>

### 考察
この銘柄の場合ボックス相場の期間がないので不適。<br>
2σを超えてもブレークどころかすぐに戻されている。<br>
とはいえ、こちらもすぐ反応はしていない。負の相関が心持ち見える？<br>

In [ ]:
fig,ax1=plt.subplots(figsize=(16,4))
ax2=plt.twinx(ax1)

# 描画する
ax1.plot(d.Date, d.Close_adj, label="株価", color="blue")

std = d.Close_adj.rolling(25).std()
mean = d.Close_adj.rolling(25).mean()
d["乖離度"] = (d.Close_adj-mean) / std
ax2.plot(d.Date, d["乖離度"], label="乖離度",color="red")
ax1.plot(d.Date, mean, label="25日移動平均線", color="orange")
ax1.plot(d.Date, mean + std, label="+1σ")
ax1.plot(d.Date, mean - std, label="-1σ")
ax1.plot(d.Date, mean + 2*std, label="+2σ")
ax1.plot(d.Date, mean - 2*std, label="-2σ")

s1 = d["乖離度"] > 2
s2 = d["乖離度"] < -2

d["2σクロス"] = np.where(s1 & ~s1.shift().fillna(False), 1,
                         np.where(s2 & ~s2.shift().fillna(False), -1, 0)
                     )
ax2.plot(d.Date,d["2σクロス"], label="ゴールデン・デッドクロス")


# グラフ周りの設定
plt.title('株価推移')
ax1.set_xlabel('年月')
ax1.set_ylabel('株価')
ax2.set_ylabel('Z値(/σ)')

plt.grid()

handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
ax1.legend(handler1 + handler2, label1 + label2)

plt.show()

sns.scatterplot(x="乖離度", y="Target", data=d)
plt.show()


### シミュレーション
1.順張戦略 <br>
たとえば2σ超えたら（ゴールデンクロスのように、1日前は2σ未満、当日2σ以上の場合）順張りを入れる<br>
※ 期間が短いため、標準偏差算出は25日を基準とする。<br>
  パラメタ検証のため、しきい値は1σ-3σまで変えてみる<br>
<br>
2.逆張り戦略<br>
１と逆の売り買いすればよいだけなので割愛<br>

In [ ]:
def simulate(d, threshold=2):
    series = d.Close_adj
    target = d.Target
    std = series.rolling(25).std()
    mean = series.rolling(25).mean()
    a = (series-mean) / std

    s1 = a > threshold
    s2 = a < -threshold

    s = pd.Series(np.where(s1 & ~s1.shift().fillna(False), 1,
                             np.where(s2 & ~s2.shift().fillna(False), -1, 0)
                         ), index=series.index, name="Cross")

    data =  pd.concat([s, target], axis=1).groupby("Cross")[target.name].sum()
    return data[data.index>0].sum() - data[data.index<0].sum()
#simulate(d)
result = {}
for i in range(5):
    result[i] = df.groupby("SecuritiesCode").apply(lambda _d: simulate(_d, 1 + 0.5*i ))

In [ ]:
import scipy.stats as stats

for i in range(5):
    print(i,(result[i]!=0).sum())
    print("合計",result[i].sum())
    print("平均",result[i][result[i]!=0].mean())
    result[i][result[i]!=0].hist(bins=np.arange(50)/100 - 0.25)
    plt.show()
    
    fig = plt.subplots(figsize=(8,8)) 
    stats.probplot(result[i][result[i]!=0], dist="norm", plot=plt)
    plt.show()
    print("S-W検定",stats.shapiro(result[i]))
    print("K-S検定",stats.kstest(result[i], "norm"))


### 検証
統計的に有意な差かどうか検証。上記から正規分布とは仮定できそうにない。<br>
ノンパラメトリックに勝ち負けで考えてみる。<br>
2000銘柄で、正（勝ち）、負（負け）が偶然起こる範囲かどうか。<br>
ウィルコクソンの符号付き順位和検定<br>
明らかに順張りは負けている＝逆張りの方が良い。　有意水準を 0.01/5 としても有意。<br>

In [ ]:
for i in range(5):
    print(i, result[i][result[i]!=0].mean())
    print(stats.wilcoxon(result[i], y=None, zero_method='wilcox', correction=False))

### 結論
本来の用い方とは異なるが、短期的（少なくとも当日）には逆張り戦略の方が良い<br>
期待値を見ると1σ～2σの間ぐらいに最適値がありそうだが、銘柄ごとに本来は異なるだろう。<br>
ただ、この期間のデータ量で個別にパラメタ最適化は難しそう。<br>

## RSI
### 計算式
$$
    　RSI = \frac{A}{A+B}
$$
<br>
<br>
A:値上がり幅平均、B：値下がり幅平均<br>
n=14と仮設定<br>
個別銘柄での有効性は高くないが参考として。
一般には25～75％ぐらいから離れ過ぎと判断し、一度超えた後戻ってきたときにそのまま50付近まで戻るパターンが多い。

### 考察
・この銘柄では30－70程度をうろついていて、そのあたりが指標となりそう。<br>
・30を下回ってからまた30を超えた時、逆に70を超えた後下回ったときに戻る傾向にある。<br>

In [ ]:
def rsi(series, n=14):
    return (series - series.shift(1)).rolling(n).apply(lambda s:s[s>0].sum()/abs(s).sum())

fig,ax1=plt.subplots(figsize=(16,4))
ax2=plt.twinx(ax1)

# 描画する
ax1.plot(d.Date, d.Close_adj, label="株価", color="blue")
ax1.plot(d.Date, d.Close_adj.rolling(25).mean(), label="25日移動平均線", color="orange")

d["RSI"] = rsi(d.Close_adj)
ax2.plot(d.Date, d["RSI"], label="RSI(14日)", color="red")
# s = stochastic(d.Close_adj)
# ax2.plot(d.Date, s[0], label="%K(5日)")
# ax2.plot(d.Date, s[1], label="%D(3)")
# ax2.plot(d.Date, s[2], label="SLOW-%D(3)")


# グラフ周りの設定
plt.title('株価推移')
ax1.set_xlabel('年月')
ax1.set_ylabel('株価')
ax2.set_ylabel('RSI')

# gridの設定
plt.grid()
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
ax1.legend(handler1 + handler2, label1 + label2)
plt.show()

print("RSI（14日）　統計量")
display(d["RSI"].describe())
d["RSI"].plot.box()
plt.show()

sns.scatterplot(x="RSI", y="Target", data=d)
plt.show()


### シミュレーション
RSIが70%超えたあと、下回ったら売り、
逆に30%を下回ったあと、また超えたら買い。<br>
※ RSI算出期間はとりあえずn=14とする<br>

In [ ]:
def simulate(d, threshold=0.2, n=14):
    series = d.Close_adj
    target = d.Target
    a = rsi(series,n=n)

    s1 = a < 0.5 - threshold
    s2 = a > 0.5 + threshold

    s = pd.Series(np.where(~s1 & s1.shift().fillna(False), 1,
                             np.where(~s2 & s2.shift().fillna(False), -1, 0)
                         ), index=series.index, name="BuySell")

    data =  pd.concat([s, target], axis=1).groupby("BuySell")[target.name].sum()
    return data[data.index>0].sum() - data[data.index<0].sum()
#simulate(d)
result = {}
for i in range(5):
    print(i)
    result[i] = df.groupby("SecuritiesCode").apply(lambda _d: simulate(_d, threshold=0.1 + 0.05*i ))

In [ ]:
import scipy.stats as stats

for i in range(5):
    print(i,(result[i]!=0).sum())
    print("合計",result[i].sum())
    print("平均",result[i][result[i]!=0].mean())
    result[i][result[i]!=0].hist(bins=np.arange(50)/100 - 0.25)
    plt.show()
    
    fig = plt.subplots(figsize=(8,8)) 
    stats.probplot(result[i][result[i]!=0], dist="norm", plot=plt)
    plt.show()
    print("S-W検定",stats.shapiro(result[i]))
    print("K-S検定",stats.kstest(result[i], "norm"))


### 検証
統計的に有意な差かどうか検証。上記から正規分布とは仮定できそうにない。<br>
ノンパラメトリックに勝ち負けで考えてみる。<br>
2000銘柄で、正（勝ち）、負（負け）が偶然起こる範囲かどうか。<br>
ウィルコクソンの符号付き順位和検定<br>
一般的な70％付近がやはり良さそう。<br>
それでなくても全体的に勝てている。有意水準を 0.01/5 としても有意。<br>

In [ ]:
for i in range(5):
    print(i, result[i][result[i]!=0].mean())
    print(stats.wilcoxon(result[i], y=None, zero_method='wilcox', correction=False))

## ストキャスティクス
RSI 同様の考えで、現在の株価が直近に対してどの位置にあるかを表す。<br>
%K, %D（SLOW-%K）, SLOW-%Dの順で移動平均による平滑化が行われるため、移動平均と同様にデッドクロス、ゴールデンクロスを判断する。<br>
%Kは5,9,14日あたりが一般的。短いほどトレンド転換の初動を捉えるが、騙しに引っかかることも多くなる。<br>

#### 買シグナル
・一番反応が遅いSLOW-%Dが0－20％ぐらいの位置にある時は売られすぎとして買い <br>
・％DがSLOW-%Dを上に突き抜けたときは買い<br>
※売りシグナルはその逆<br>
### 考察
k=5では明らかに騙しに引っかかっている。他のオシレータと合わせて用いるべき。単体ではあまり使えそうにない。<br>
パラメタチューニングが重要そうだがこのデータ数では困難か？<br>

In [ ]:
def stochastic(series, k=5, n=3, m=3):
    _min = series.rolling(k).min()
    _max = series.rolling(k).max()
    _k = (series - _min)/(_max - _min)
    _d1 = _k.rolling(n).mean()
    _d2 = _d1.rolling(m).mean()
    return _k, _d1, _d2

fig,ax1=plt.subplots(figsize=(16,4))
ax2=plt.twinx(ax1)

# 描画する
ax1.plot(d.Date, d.Close_adj, label="株価", color="blue")
# ax1.plot(d.Date, d.Close_adj.rolling(25).mean(), label="25日移動平均線", color="orange")

_k, _d1, _d2 = stochastic(d.Close_adj)
ax2.plot(d.Date, _k, label="%K(5日)")
ax2.plot(d.Date, _d1, label="%D(3)")
ax2.plot(d.Date, _d2, label="SLOW-%D(3)")

s = (_d1 > _d2).astype(np.int8)
s = s - s.shift(1)

threshold=0.3
s = pd.Series(np.where((_d2<0.5-threshold) | (s>0), 1,
                         np.where((_d2>0.5+threshold) | (s<0), -1, 0)
                     ), index=d.index, name="BuySell")

ax2.plot(d.Date, s, label="シグナル")


# グラフ周りの設定
plt.title('株価推移')
ax1.set_xlabel('年月')
ax1.set_ylabel('株価')
ax2.set_ylabel('ストキャスティクス')

# gridの設定
plt.grid()
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
ax1.legend(handler1 + handler2, label1 + label2,bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


### シミュレーション
買シグナル<br>
・一番反応が遅いSLOW-%Dが0－20％ぐらいの位置にある時は売られすぎとして買い <br>
・％DがSLOW-%Dを上に突き抜けたときは買い<br>
※売りシグナルはその逆<br>
少し長めの日数、k=14としてシミュレート<br>

In [ ]:
%%time
def simulate(d, threshold=0.3, k=14):
    series = d.Close_adj
    target = d.Target
    _, _k, _d = stochastic(series,k=k)

    s = (_k > _d).astype(np.int8)
    s = s - s.shift(1)

    s = pd.Series(np.where((_d<0.5-threshold) | (s>0), 1,
                             np.where((_d>0.5+threshold) | (s<0), -1, 0)
                         ), index=series.index, name="BuySell")
    data =  pd.concat([s, target], axis=1).groupby("BuySell")[target.name].sum()
    return data[data.index>0].sum() - data[data.index<0].sum()
#simulate(d)
result = {}
for i in range(5):
    print(i)
    result[i] = df.groupby("SecuritiesCode").apply(lambda _d: simulate(_d, threshold=0.2 + 0.05*i ))

In [ ]:
import scipy.stats as stats

for i in range(5):
    print(i,(result[i]!=0).sum())
    print("合計",result[i].sum())
    print("平均",result[i][result[i]!=0].mean())
    result[i][result[i]!=0].hist(bins=np.arange(50)/100 - 0.25)
    plt.show()
    
    fig = plt.subplots(figsize=(8,8)) 
    stats.probplot(result[i][result[i]!=0], dist="norm", plot=plt)
    plt.show()
    print("S-W検定",stats.shapiro(result[i]))
    print("K-S検定",stats.kstest(result[i], "norm"))


### 検証
統計的に有意な差かどうか検証。上記から正規分布とは仮定できそうにない。<br>
ノンパラメトリックに勝ち負けで考えてみる。<br>
2000銘柄で、正（勝ち）、負（負け）が偶然起こる範囲かどうか。<br>
ウィルコクソンの符号付き順位和検定<br>
しきい値を90％にしてしまうとそもそも売り買いできる銘柄が減ってしまうのでいまいち。勝率も特に高くならない。<br>
一般的な80％かもう少し緩めて70％付近でも良いかもしれない。<br>
ただ全体的に勝てているので有効な指標となる。有意水準を 0.01/5 としても有意。<br>

In [ ]:
for i in range(5):
    print(i, result[i][result[i]!=0].mean())
    print(stats.wilcoxon(result[i], y=None, zero_method='wilcox', correction=False))